In [1]:
# Training script for LazyConfig models
try:
    # ignore ShapelyDeprecationWarning from fvcore
    import warnings

    from shapely.errors import ShapelyDeprecationWarning

    warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

except:
    pass
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger

setup_logger()

import gc
import os
import time

import detectron2.utils.comm as comm

# import some common libraries
import numpy as np
import torch

# import some common detectron2 utilities
from detectron2.config import LazyConfig, get_cfg
import detectron2.data as data
from detectron2.engine import (
    launch,
)

from deepdisc.data_format.augment_image import train_augs
from deepdisc.data_format.image_readers import DC2ImageReader
from deepdisc.data_format.register_data import register_data_set#, register_loaded_data_set
from deepdisc.model.loaders import (
    RedshiftFlatDictMapper,
    return_test_loader,
    return_train_loader,
)
from deepdisc.model.models import RedshiftPointCasROIHeads, RedshiftPointROIHeads, RedshiftPDFROIHeads, return_lazy_model
from deepdisc.training.trainers import (
    return_evallosshook,
    return_lazy_trainer,
    return_optimizer,
    return_savehook,
    return_schedulerhook,
)
from deepdisc.utils.parse_arguments import make_training_arg_parser



In [2]:
#import importlib
#importlib.reload(rail.estimation.algos.deepdisc)

In [3]:
#from rail.estimation.algos.deepdisc import DeepDiscInformer
import rail
from rail.estimation.algos.deepdisc import *
from rail.core.data import TableHandle
from rail.core.stage import RailStage

from rail.deepdisc.configs import *

In [4]:
cfgfile = "/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py"
dirpath = './tests/deepdisc/test_data/dc2/'
output_dir = './'
output_name = 'test'

trainfile = dirpath + "flattened_data_test.npy"
testfile = dirpath + "flattened_data_test.npy"
classes = ['object']
numclasses = 1

In [5]:
cfg = get_lazy_config(cfgfile, 1, 1)
cfg.train.init_checkpoint

'detectron2://ImageNetPretrained/MSRA/R-50.pkl'

In [6]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [7]:
testdata = np.load('./tests/deepdisc/test_data/dc2/flattened_data_test.npy')
mapper = RedshiftFlatDictMapper().map_data

In [8]:
dataset_dicts={}
dds = []
for row in testdata:
    dds.append(mapper(row))
dataset_dicts['test'] = dds

In [9]:
training = DS.add_data("training", testdata, TableHandle)
testing = DS.add_data("testing", testdata, TableHandle)

In [10]:
deep_dict = dict(epochs=200,
                numclasses=1,
                batch_size=1,
                output_dir ='./',
                cfgfile="/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py",
                output_name ='test_informer')

In [11]:
#Inform = DeepDiscInformer.make_stage(name='Inform_DeepDISC', model='detectron2://ImageNetPretrained/MSRA/R-50.pkl', **deep_dict)
Inform = DeepDiscInformer.make_stage(name='Inform_DeepDISC', model='test_informer.pkl', **deep_dict)


In [12]:
Inform.inform(training)

[12/01 16:04:36 d2.data.build]: Making batched data loader with batch_size=1
[12/01 16:04:36 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from detectron2://ImageNetPretrained/MSRA/R-50.pkl ...
[12/01 16:04:36 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
WARNING [12/01 16:04:36 d2.checkpoint.c2_model_loading]: Shape of stem.conv1.weight in checkpoint is torch.Size([64, 3, 7, 7]), while shape of backbone.bottom_up.stem.conv1.weight in model is torch.Size([64, 6, 7, 7]).
WARNING [12/01 16:04:36 d2.checkpoint.c2_model_loading]: stem.conv1.weight will not be loaded. Please double check and see if this is desired.
[12/01 16:04:36 d2.checkpoint.c2_model_loading]: Following weights matched with submodule backbone.bottom_up - Total num: 53


Some model parameters or buffers are not found in the checkpoint:
backbone.bottom_up.stem.conv1.weight
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.deconv.{bias, weight}
roi_heads.mask_head.mask_fcn1.{bias, weight}
roi_heads.mask_head.mask_fcn2.{bias, weight}
roi_heads.mask_head.mask_fcn3.{bias, weight}
roi_heads.mask_head.mask_fcn4.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight

Model training:
[12/01 16:04:36 d2.engine.train_loop]: Starting training from iteration 0
Iteration:  5  time:  3.109453245997429e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.08168353140354156, 0.03492044284939766, 0.09005818186712308, 0.5837008953094482, 0.0014377956977114081] val loss:  0 lr:  [0.001]
Iteration:  10  time:  4.341127350926399e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.13382689654827118, 0.028655963018536568, 0.07374486869537594, 0.2618156671524048, 0.001036638393998146] val loss:  0.7522368614356524 lr:  [0.001]
Iteration:  15  time:  5.069887265563011e-07 dict_keys(['loss_cls', 'loss_box_reg', 'redshift_loss', 'loss_rpn_cls', 'loss_rpn_loc']) [0.04372570291161537, 0.003985135350376368, 0.08474627610075508, 0.06478501111268997, 0.0014642772730439901] val loss:  0.33172891637948587 lr:  [0.001]
Iteration:  20  time:  4.019821062684059e-07 dict_keys(['loss_cls', 'loss

### Inference

In [13]:
#Estimator = DeepDiscEstimator.make_stage(name='DeepDiscEstimator',
#                                       model=Inform.get_handle('model'), **deep_dict)

Estimator = DeepDiscPDFEstimator.make_stage(name='DeepDiscEstimator',
                                       model=Inform.get_handle('model'), hdf5_groupname=None, **deep_dict)

In [14]:

results = Estimator.estimate(testing)



[12/01 16:06:19 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./test_informer.pth ...
Processing Data
Matching objects
[] []


ValueError: Shape of xbins in xvals (200) != shape of xbins in yvals (0)

In [15]:
res = results.read()

In [16]:
res